# Лабораторная работа 2, вторая часть

## Португальский и французский языки

Во второй части работы исследуются свойства кросс-языкового отображения. 

Загружаем необходимые библиотеки:

In [1]:
import  gensim
import  numpy as  np
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings('ignore')

Теперь загружаем векторные представления слов португальского и французского языков:

In [2]:
pt_emb = KeyedVectors.load_word2vec_format("cc.pt.300.vec",binary=False, limit = 10000)
fr_emb = KeyedVectors.load_word2vec_format("cc.fr.300.vec",binary=False, limit = 10000)

### Перевод с португальского на французский:

Сначала загрузим пары слов (португальский, французский) из текстовых файлов

In [3]:
def load_word_pairs(filename):
    pt_fr_pairs = []
    fr_vectors = []
    pt_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            pt, fr = line.rstrip().split("\t")
            if fr not in fr_emb or pt not in pt_emb:
                continue
            pt_fr_pairs.append((pt, fr))
            fr_vectors.append(fr_emb[fr])
            pt_vectors.append(pt_emb[pt])
            
    return pt_fr_pairs, np.array(pt_vectors), np.array(fr_vectors)

In [4]:
pt_fr_train, X_train, Y_train = load_word_pairs("pt-fr.0-5000.txt")

In [5]:
pt_fr_test, X_test, Y_test = load_word_pairs("pt-fr.5000-6500.txt")

In [6]:
def learn_transform(X, Y):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

Строим матрицы отображений между пространствами векторных репрезентаций:

In [8]:
M1 = learn_transform(X_train[:300], Y_train[:300])
M2 = learn_transform(X_train[300:601], Y_train[300:601])

In [9]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE   
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in fr_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [10]:
print("pair1 M1: {}".format(precision(pt_fr_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(pt_fr_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(pt_fr_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(pt_fr_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.7066666666666667
pair1 M2: 0.7066666666666667
pair2 M1: 0.7508305647840532
pair2 M2: 0.7508305647840532


Подсчитали точность перевода языковых пар для каждой из матриц:

In [11]:
M1_M2 = np.linalg.norm((M1-M2),ord='fro')
M1_M2

0.0

In [12]:
M1_M2/np.linalg.norm(M1,ord='fro')

0.0

Норма Фробениуса для разности матриц равна нулю.

Теперь проверим каждую из матриц $M_1$ и $M_2$ на ортогональность. Для этого вычислим произведения $M_1 \cdot M_1 ^ T$ и $M_2 \cdot M_2 ^ T$. 

In [13]:
print(np.matmul(M1,M1.T))

[[ 1.0000001e+00 -2.9802322e-08  3.7252903e-09 ... -1.8859282e-08
  -2.9802322e-08 -1.6763806e-08]
 [-2.9802322e-08  9.9999976e-01 -2.9336661e-08 ...  9.3132257e-09
  -3.8184226e-08 -3.3527613e-08]
 [ 3.7252903e-09 -2.9336661e-08  9.9999976e-01 ...  1.8626451e-09
  -9.3132257e-09 -3.1664968e-08]
 ...
 [-1.8859282e-08  9.3132257e-09  1.8626451e-09 ...  1.0000000e+00
  -2.2351742e-08 -6.5192580e-09]
 [-2.9802322e-08 -3.8184226e-08 -9.3132257e-09 ... -2.2351742e-08
   1.0000000e+00  9.3132257e-09]
 [-1.6763806e-08 -3.3527613e-08 -3.1664968e-08 ... -6.5192580e-09
   9.3132257e-09  1.0000000e+00]]


In [14]:
print(np.matmul(M2,M2.T))

[[ 1.0000001e+00 -2.9802322e-08  3.7252903e-09 ... -1.8859282e-08
  -2.9802322e-08 -1.6763806e-08]
 [-2.9802322e-08  9.9999976e-01 -2.9336661e-08 ...  9.3132257e-09
  -3.8184226e-08 -3.3527613e-08]
 [ 3.7252903e-09 -2.9336661e-08  9.9999976e-01 ...  1.8626451e-09
  -9.3132257e-09 -3.1664968e-08]
 ...
 [-1.8859282e-08  9.3132257e-09  1.8626451e-09 ...  1.0000000e+00
  -2.2351742e-08 -6.5192580e-09]
 [-2.9802322e-08 -3.8184226e-08 -9.3132257e-09 ... -2.2351742e-08
   1.0000000e+00  9.3132257e-09]
 [-1.6763806e-08 -3.3527613e-08 -3.1664968e-08 ... -6.5192580e-09
   9.3132257e-09  1.0000000e+00]]


Видим, что все элементы, кроме диагональных, очень близки к нулю, а диагональные элементы очень близки к единице. 

Значит, можно считать, что $M_1 \cdot M_1 ^ T = E$ и $M_2 \cdot M_2 ^ T = E$ и каждая из матриц $M_1, M_2$ ортогональна.

### Перевод с французского на португальский:

Теперь проделаем то же самое для пары (французский, португальский)

In [15]:
def load_word_pairs(filename):
    fr_pt_pairs = []
    fr_vectors = []
    pt_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            fr, pt = line.rstrip().split("\t")
            if fr not in fr_emb or pt not in pt_emb:
                continue
            fr_pt_pairs.append((fr, pt))
            fr_vectors.append(fr_emb[fr])
            pt_vectors.append(pt_emb[pt])
            
    return fr_pt_pairs, np.array(fr_vectors), np.array(pt_vectors)

In [16]:
fr_pt_train, X_train, Y_train = load_word_pairs("fr-pt.0-5000.txt")

In [17]:
fr_pt_test, X_test, Y_test = load_word_pairs("fr-pt.5000-6500.txt")

In [18]:
M1 = learn_transform(X_train[:300], Y_train[:300])

In [19]:
M2 = learn_transform(X_train[300:601], Y_train[300:601])

In [20]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE   
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in pt_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [21]:
print("pair1 M1: {}".format(precision(fr_pt_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(fr_pt_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(fr_pt_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(fr_pt_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.7566666666666667
pair1 M2: 0.7566666666666667
pair2 M1: 0.8106312292358804
pair2 M2: 0.8106312292358804


Видим, что точность перевода получилась выше, чем при переводе с французского на португальский.

In [22]:
M1_M2 = np.linalg.norm((M1-M2),ord='fro')
M1_M2

0.0

In [23]:
M1_M2/np.linalg.norm(M1,ord='fro')

0.0

Норма Фробениуса по-прежнему равна нулю.

In [24]:
print(np.matmul(M1,M1.T))

[[ 9.9999988e-01 -1.3038516e-08 -5.5879354e-09 ...  5.5879354e-09
  -7.4505806e-09 -5.1222742e-09]
 [-1.3038516e-08  1.0000001e+00  3.7252903e-08 ...  1.1175871e-08
  -3.7252903e-08  5.5879354e-09]
 [-5.5879354e-09  3.7252903e-08  1.0000000e+00 ...  4.6566129e-10
   1.6763806e-08 -1.1175871e-08]
 ...
 [ 5.5879354e-09  1.1175871e-08  4.6566129e-10 ...  1.0000000e+00
   6.5192580e-09 -2.7939677e-08]
 [-7.4505806e-09 -3.7252903e-08  1.6763806e-08 ...  6.5192580e-09
   1.0000000e+00 -2.2351742e-08]
 [-5.1222742e-09  5.5879354e-09 -1.1175871e-08 ... -2.7939677e-08
  -2.2351742e-08  1.0000000e+00]]


In [25]:
print(np.matmul(M2,M2.T))

[[ 9.9999988e-01 -1.3038516e-08 -5.5879354e-09 ...  5.5879354e-09
  -7.4505806e-09 -5.1222742e-09]
 [-1.3038516e-08  1.0000001e+00  3.7252903e-08 ...  1.1175871e-08
  -3.7252903e-08  5.5879354e-09]
 [-5.5879354e-09  3.7252903e-08  1.0000000e+00 ...  4.6566129e-10
   1.6763806e-08 -1.1175871e-08]
 ...
 [ 5.5879354e-09  1.1175871e-08  4.6566129e-10 ...  1.0000000e+00
   6.5192580e-09 -2.7939677e-08]
 [-7.4505806e-09 -3.7252903e-08  1.6763806e-08 ...  6.5192580e-09
   1.0000000e+00 -2.2351742e-08]
 [-5.1222742e-09  5.5879354e-09 -1.1175871e-08 ... -2.7939677e-08
  -2.2351742e-08  1.0000000e+00]]


Кроме того, обе матрицы ортогональны.